# Clustering-based Customer Segmentation

## 📦 1. Install Required Libraries

In [1]:
!pip install -q kneed openpyxl

## 📚 2. Import Libraries

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from kneed import KneeLocator
from collections import defaultdict
import pickle

## 📥 3. Load Processed Data

In [4]:
df = pd.read_excel('processed_data_final.xlsx')
df

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_encoded,main_category,main_category_encoded,day_of_week,month,hour,day_of_year
0,2019-10-08 05:16:23,view,1004825,2053013555631883008,electronics.smartphone,bq,39.36,547394949,c3a4ded7-ae7e-9529-9453-64e8e3ef0fb6,2,electronics,7,1,10,5,281
1,2019-10-04 05:56:37,view,2501352,2053013564003714048,appliances.kitchen.oven,beko,257.38,513147415,339c68b1-7e6b-47cc-bd4f-3049c82779dc,2,appliances,2,4,10,5,277
2,2019-10-04 18:35:44,view,1003312,2053013555631883008,electronics.smartphone,apple,703.52,513815293,7196708c-3af6-45dd-a6e6-6f2d4b271505,2,electronics,7,4,10,18,277
3,2019-10-02 12:59:07,view,1005018,2053013555631883008,electronics.smartphone,huawei,95.82,512388419,297154b9-3db5-4f59-9113-acfd61d7db28,2,electronics,7,2,10,12,275
4,2019-10-13 14:32:18,view,4201166,2053013552351937024,appliances.environment.air_conditioner,tcl,295.22,512771890,2eff34cc-a57e-4a79-8cc3-c5d803f5ac8b,2,appliances,2,6,10,14,286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18375,2019-10-09 11:49:19,cart,1004767,2053013555631883008,electronics.smartphone,samsung,250.93,550358899,6ec714ee-dca5-4e79-9242-0f73ca4ba780,0,electronics,7,2,10,11,282
18376,2019-10-31 19:36:07,view,1004777,2053013555631883008,electronics.smartphone,xiaomi,135.01,566236354,db2f17a6-5f6e-4a0c-9c7d-9141a7b4b788,2,electronics,7,3,10,19,304
18377,2019-10-26 02:29:37,view,1004766,2053013555631883008,electronics.smartphone,samsung,244.46,545262548,1c59b85c-ccb7-4fb7-bc6c-5944869faf26,2,electronics,7,5,10,2,299
18378,2019-10-27 19:29:52,view,1004872,2053013555631883008,electronics.smartphone,samsung,278.40,564497571,da66d02f-bacc-4e0e-b8e6-b5e4badee0cc,2,electronics,7,6,10,19,300


## 🧮 4. Aggregate User-Level Features

In [5]:
user_features = df.groupby('user_id').agg(
    activity_frequency=('event_type', 'count'),
    category_count=('main_category', 'nunique'),
    event_type_count=('event_type', 'nunique'),
    unique_products=('product_id', 'nunique'),
    unique_brands=('brand', 'nunique'),
    average_spend=('price', 'mean')
).reset_index()

## ⚖️ 5. Standardize the Features

In [6]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(user_features.drop(columns=['user_id']))

## 🤖 6. Apply KMeans Clustering (Best Parameters)

In [7]:
kmeans = KMeans(
    n_clusters=3,
    init='k-means++',
    n_init=20,
    max_iter=300,
    algorithm='lloyd',
    random_state=42
)
kmeans_labels = kmeans.fit_predict(scaled_data)
user_features['final_cluster'] = kmeans_labels

## 💾 7. Save StandardScaler for Future Use

In [8]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)


## 🧩 8. Add Descriptions to Each Cluster

In [10]:
segment_descriptions = {
    0: "Single High-Value Users - One-time buyers with high spend.",
    1: "Active Loyal Users - Frequent buyers in a single category.",
    2: "Occasional Explorers - Moderate activity across categories."
}

for i, group in user_features.groupby('final_cluster'):
    print(f"\nCluster {i}: {segment_descriptions[i]}")
    print(group.describe(include='all'))


Cluster 0: Single High-Value Users - One-time buyers with high spend.
            user_id  activity_frequency  category_count  event_type_count  \
count  1.760300e+04        17603.000000         17603.0           17603.0   
mean   5.340297e+08            1.001250             1.0               1.0   
std    1.864296e+07            0.035331             0.0               0.0   
min    3.015733e+08            1.000000             1.0               1.0   
25%    5.160905e+08            1.000000             1.0               1.0   
50%    5.305083e+08            1.000000             1.0               1.0   
75%    5.524036e+08            1.000000             1.0               1.0   
max    5.662385e+08            2.000000             1.0               1.0   

       unique_products  unique_brands  average_spend  final_cluster  
count          17603.0        17603.0   17603.000000        17603.0  
mean               1.0            1.0     268.312056            0.0  
std                0.0   

## 🔁 9. Merge Cluster Labels Back into Original DataFrame

In [11]:
df = df.merge(user_features[['user_id', 'final_cluster']], on='user_id', how='left')
df


,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_encoded,main_category,main_category_encoded,day_of_week,month,hour,day_of_year,final_cluster
0,2019-10-08 05:16:23,view,1004825,2053013555631883008,electronics.smartphone,bq,39.36,547394949,c3a4ded7-ae7e-9529-9453-64e8e3ef0fb6,2,electronics,7,1,10,5,281,0
1,2019-10-04 05:56:37,view,2501352,2053013564003714048,appliances.kitchen.oven,beko,257.38,513147415,339c68b1-7e6b-47cc-bd4f-3049c82779dc,2,appliances,2,4,10,5,277,0
2,2019-10-04 18:35:44,view,1003312,2053013555631883008,electronics.smartphone,apple,703.52,513815293,7196708c-3af6-45dd-a6e6-6f2d4b271505,2,electronics,7,4,10,18,277,0
3,2019-10-02 12:59:07,view,1005018,2053013555631883008,electronics.smartphone,huawei,95.82,512388419,297154b9-3db5-4f59-9113-acfd61d7db28,2,electronics,7,2,10,12,275,1
4,2019-10-13 14:32:18,view,4201166,2053013552351937024,appliances.environment.air_conditioner,tcl,295.22,512771890,2eff34cc-a57e-4a79-8cc3-c5d803f5ac8b,2,appliances,2,6,10,14,286,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18375,2019-10-09 11:49:19,cart,1004767,2053013555631883008,electronics.smartphone,samsung,250.93,550358899,6ec714ee-dca5-4e79-9242-0f73ca4ba780,0,electronics,7,2,10,11,282,0
18376,2019-10-31 19:36:07,view,1004777,2053013555631883008,electronics.smartphone,xiaomi,135.01,566236354,db2f17a6-5f6e-4a0c-9c7d-9141a7b4b788,2,electronics,7,3,10,19,304,0
18377,2019-10-26 02:29:37,view,1004766,2053013555631883008,electronics.smartphone,samsung,244.46,545262548,1c59b85c-ccb7-4fb7-bc6c-5944869faf26,2,electronics,7,5,10,2,299,0
18378,2019-10-27 19:29:52,view,1004872,2053013555631883008,electronics.smartphone,samsung,278.40,564497571,da66d02f-bacc-4e0e-b8e6-b5e4badee0cc,2,electronics,7,6,10,19,300,0


## 🏷️ 10. Create and Reorder Subcategory Column

In [12]:
# Step 1: Create 'sub_category' column
df['sub_category'] = df['category_code'].str.split('.').str[1]

# Step 2: Reorder columns to insert 'sub_category' after 'main_category_encoded'
cols = list(df.columns)
if 'sub_category' in cols:
    cols.remove('sub_category')

if 'main_category_encoded' in cols:
    idx = cols.index('main_category_encoded')
    cols.insert(idx + 1, 'sub_category')
    df = df[cols]

# Preview updated DataFrame
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,event_type_encoded,main_category,main_category_encoded,sub_category,day_of_week,month,hour,day_of_year,final_cluster
0,2019-10-08 05:16:23,view,1004825,2053013555631883008,electronics.smartphone,bq,39.36,547394949,c3a4ded7-ae7e-9529-9453-64e8e3ef0fb6,2,electronics,7,smartphone,1,10,5,281,0
1,2019-10-04 05:56:37,view,2501352,2053013564003714048,appliances.kitchen.oven,beko,257.38,513147415,339c68b1-7e6b-47cc-bd4f-3049c82779dc,2,appliances,2,kitchen,4,10,5,277,0
2,2019-10-04 18:35:44,view,1003312,2053013555631883008,electronics.smartphone,apple,703.52,513815293,7196708c-3af6-45dd-a6e6-6f2d4b271505,2,electronics,7,smartphone,4,10,18,277,0
3,2019-10-02 12:59:07,view,1005018,2053013555631883008,electronics.smartphone,huawei,95.82,512388419,297154b9-3db5-4f59-9113-acfd61d7db28,2,electronics,7,smartphone,2,10,12,275,1
4,2019-10-13 14:32:18,view,4201166,2053013552351937024,appliances.environment.air_conditioner,tcl,295.22,512771890,2eff34cc-a57e-4a79-8cc3-c5d803f5ac8b,2,appliances,2,environment,6,10,14,286,0


## 📁 11. Save Merged Data with Clusters

In [13]:
df.to_csv('merged_data.csv', index=False)

## 🧭 12. Build Nested Dropdown Structure (Main > Sub > Brand)

In [14]:
# Load merged data
df = pd.read_csv('merged_data.csv')

# Get unique combinations of main_category, sub_category, brand
unique_combos = df[['main_category', 'sub_category', 'brand']].drop_duplicates()

# Create nested dictionary structure
dropdown_structure = defaultdict(lambda: defaultdict(set))
for _, row in unique_combos.iterrows():
    dropdown_structure[row['main_category']][row['sub_category']].add(row['brand'])

# Convert sets to sorted lists
nested_dropdowns = {
    main_cat: {
        sub_cat: sorted(list(brands))
        for sub_cat, brands in sub_dict.items()
    }
    for main_cat, sub_dict in dropdown_structure.items()
}

# ✅ Now nested_dropdowns is ready to use for UI dropdowns
nested_dropdowns


{'electronics': {'smartphone': ['apple',
   'asus',
   'bq',
   'doogee',
   'fly',
   'gionee',
   'google',
   'haier',
   'honor',
   'huawei',
   'inoi',
   'jinga',
   'leeco',
   'lg',
   'meizu',
   'nokia',
   'nubia',
   'oneplus',
   'oppo',
   'prestigio',
   'samsung',
   'sony',
   'tecno',
   'tp-link',
   'umi',
   'vivo',
   'xiaomi',
   'zte'],
  'clocks': ['acme',
   'adriatica',
   'aimoto',
   'apple',
   'armani',
   'balmain',
   'boccia',
   'canyon',
   'casio',
   'certina',
   'citizen',
   'cover',
   'danielklein',
   'diesel',
   'dkny',
   'elari',
   'fitbit',
   'fossil',
   'garmin',
   'hanowa',
   'honor',
   'huawei',
   'jet',
   'lg',
   'lorus',
   'mido',
   'mykronoz',
   'orient',
   'otan',
   'roamer',
   'romanson',
   'samsung',
   'seiko',
   'skagen',
   'suunto',
   'swatch',
   'tissot',
   'troyka',
   'versace',
   'versus',
   'wonlex',
   'xiaomi',
   'zeppelin'],
  'audio': ['acme',
   'admira',
   'akai',
   'alesis',
   'alphard'

## 13. Print out user_features.csv

In [15]:
# --- Step 1: Install Required Libraries (if needed) ---
!pip install -q openpyxl

# --- Step 2: Import Libraries ---
import pandas as pd

# --- Step 3: Load Processed Data from Excel ---
df = pd.read_excel('processed_data_final.xlsx')

# --- Step 4: Group by User and Create Aggregated Features ---
user_features = df.groupby('user_id').agg(
    activity_frequency=('event_type', 'count'),
    category_count=('main_category', 'nunique'),
    event_type_count=('event_type', 'nunique'),
    unique_products=('product_id', 'nunique'),
    unique_brands=('brand', 'nunique'),
    average_spend=('price', 'mean')
).reset_index()

# --- Step 5: Drop user_id column ---
user_features = user_features.drop(columns=['user_id'])

# --- Step 6: Save to CSV ---
user_features.to_csv('user_features.csv', index=False)

# Optional: Display the result
user_features.head()


,activity_frequency,category_count,event_type_count,unique_products,unique_brands,average_spend
0,1,1,1,1,1,420.60
1,1,1,1,1,1,39.35
2,1,1,1,1,1,189.97
3,1,1,1,1,1,529.74
4,1,1,1,1,1,33.45


In [16]:
from google.colab import files
files.download('user_features.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>